In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python


fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1AyULTkscFRSlZIJ143yXe7qg8fSNPBp-'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('util.py')


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1IugJe_XgxWtVkim24TlGz36YiUwGX13D'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('alexnet.py')

In [0]:
import sys
import os.path
import tensorflow as tf
import util as tu
import numpy as np
import alexnet as an

In [0]:
def classify(image,top_k,k_patches,ckpt_path,imagenet_path):
  """Procedure to classify the image given through the command line
    
    
    Args:
          image: path to the image to classify
          top_k : integer representing the number of predictions with highest probability
                  to retrieve
          k_patches: number of crops taken from an image and to input to the model
          ckpt_path: path to model's tensorflow checkpoint
          imagenet_path: path to ILSVRC12 ImageNet folder containing train images,
                            validation images,annotations and metadata file
  """
  wnids,words = tu.load_imagenet_meta(os.paqth.join(imagenet_path,'data/meta.mat'))
  
  
  #taking a few crops from an image
  image_patches = tu.read_k_patches(image,k_patches)
  
  x = tf.placeholder(tf.float32,[None,224,224,3])
  
  _,pred = an.classifier(x,dropout = 1.0)
  
  #calculate the average precision through the crops
  
  avg_prediction = tf.div(tf.reduce_sum(pred,0),k_patches)
  
  scores,indexes = tf.nn.top_k(avg_prediction, k =top_k)
  
  saver = tf.train.Saver()
  
  with tf.Session(config = tf.ConfigProto()) as sess:
        saver.restore(sess,os.path.join(ckpt_path,'alexnet-cnn.ckpt'))
        
        s,i = sess.run([scores,indexes],feed_dict = {x:image_patches})
        
        s,i = np.squeeze(s), np.squeeze(i)
        
        print('AlexNet saw:')
        
        for idx in range(top_k):
          print('{} - score: {}'.format(words[i[idx]],s[idx]))

In [0]:
if __name__ == '__main__':
  TOP_K = 5
  K_CROPS = 5
  IMAGENET_PATH = ''##CHeck path
  CKPT_PATH = 'ckpt-alexnet'
  
  image_path = sys.argv[1]
  
  classify(image_path,TOP_K,K_CROPS,CKPT_PATH,IMAGENET_PATH)